# Apriori Algorithm for Frequent Patterns and Association Rules

Source from https://github.com/chonyy/apriori_python


In [ ]:
from collections import defaultdict
from itertools import chain, combinations

### Prunning

In [ ]:
def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

### Scan data to get frequent patterns from candidates

In [ ]:
def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet

### Get association rules

In [ ]:
def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules

### Utilities

In [ ]:
def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

In [ ]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

#### Does getUnion do exactly what we have discussed in class?

In [ ]:
def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

### Apriori Algorithm 

In [ ]:
def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result, global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    print('Frequent 1 itemsets:')
    print(L1ItemSet)
    input()
    
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        print('--- Analysis for k =', k, '---')
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        print('Candidates after self-join:')
        print(candidateSet)
        input()
        
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        print('Candidates after prunning:')
        print(candidateSet)
        input()
        
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(candidateSet, itemSetList, minSup, globalItemSetWithSup)
        print('Frequent patterns after scan:')
        print(currentLSet)
        input()
        
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

### Example 1: Alphabet itemsets

In [ ]:
itemSetList = [['A', 'C', 'D'],
                ['B', 'C', 'E'],
                ['A', 'B', 'C', 'E'],
                ['B', 'E']]

In [ ]:
freqItemSet, rules = apriori(itemSetList, minSup=0.5, minConf=0.5)

In [ ]:
print(freqItemSet)

In [ ]:
print(rules)

### Example 2: Grocery store market basket

In [ ]:
itemSetList = [['eggs', 'bacon', 'soup'],
                ['eggs', 'bacon', 'apple'],
                ['soup', 'bacon', 'banana']]

In [ ]:
freqItemSet, rules = apriori(itemSetList, minSup=0.5, minConf=0.5)

In [ ]:
print(freqItemSet)

In [ ]:
print(rules)